In [1]:
import pandas as pd

In [3]:
data = pd.read_csv("data/gemstone.csv")
data.drop(["id"],axis = 1,inplace=True)
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [5]:
X = data.drop(["price"],axis=1)
Y = data[["price"]]

In [6]:
num_col = X.select_dtypes(exclude="object")
cat_col = X.select_dtypes(include="object")

In [7]:
cut_map = ["Fair"  , "Good"  , "Very Good"  , "Premium"  , "Ideal" ]
color_map = ["D"  , "E"  , "F"  , "G"  , "H" ,"I","J"]
clar_map = ["I1"  , "SI2"  , "SI1"  , "VS2"  , "VS1" ,"VVS2","VVS1","IF"]

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer # Handling missing values
from sklearn.preprocessing import OrdinalEncoder , StandardScaler
from sklearn.compose import ColumnTransformer

In [21]:
num_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy="median")),
        ('scaler', StandardScaler())

    ]
)

In [22]:
cat_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy="most_frequent")),
        ('encoder' , OrdinalEncoder(categories=[cut_map,color_map,clar_map])),
        ('scaler', StandardScaler())

    ]
)

In [27]:
preprocessor = ColumnTransformer([
    ("num_pipeline",num_pipeline,num_col.columns),
    ("cat_pipeline",cat_pipeline,cat_col.columns),
])

In [28]:
from sklearn.model_selection import train_test_split
x_train , x_test ,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=30)

In [29]:
x_train = pd.DataFrame(preprocessor.fit_transform(x_train),columns=preprocessor.get_feature_names_out())
x_test = pd.DataFrame(preprocessor.transform(x_test),columns=preprocessor.get_feature_names_out())

In [20]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [30]:
x_train

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,0.874076,1.528722,1.352731
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,-2.144558,-0.935071,-0.646786
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,-0.132136,0.296826,0.686225
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,-0.132136,0.296826,0.019720
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,0.874076,2.144670,1.352731
...,...,...,...,...,...,...,...,...,...
135496,-1.040295,-0.016876,-0.642862,-1.268122,-1.244270,-1.239078,0.874076,-0.935071,-0.646786
135497,0.991842,0.168176,-0.642862,1.048629,1.114501,1.079486,0.874076,0.296826,-1.313292
135498,0.451380,1.556060,-0.642862,0.516768,0.588314,0.702719,-2.144558,0.296826,-0.646786
135499,0.667565,-1.774863,1.442462,0.868337,0.951202,0.688228,0.874076,0.296826,0.686225


In [31]:
from sklearn.linear_model import LinearRegression

In [33]:
regre = LinearRegression()
regre.fit(x_train,y_train)
y_pred = regre.predict(x_test)

In [37]:
from sklearn.metrics import r2_score , mean_squared_error , mean_absolute_error
import numpy as np

In [40]:
print(r2_score(y_test,y_pred))
print(np.sqrt(mean_squared_error(y_test,y_pred)))

0.9368908248567511
1013.9047094344005
